In [ ]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz 
!tar xf spark-3.3.0-bin-hadoop2.tgz

In [ ]:
# configurar as variáveis de ambiente e o Spark
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop2.tgz"

In [ ]:
!pip install -q findspark

# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.3.0-bin-hadoop2')

In [ ]:
# iniciar uma sessão local
from pyspark.sql import SparkSession,SQLContext
import matplotlib.pyplot as plt

spark = SparkSession.builder.master("local[*]").appName("DadosOlist").getOrCreate()

In [ ]:
#Puxando planilhas diretamente da kaggle
from google.colab import files
files.upload()#Escolha o seu Kaggle.json (https://www.kaggle.com/discussions/general/74235)

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"humbertomarriel","key":"6f58ace884f37e2e2b80db5e29d3ac9e"}'}

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [ ]:
!kaggle datasets download -d olistbr/brazilian-ecommerce # para obter os CSVs da olist diretamente da Kaggle

 63% 27.0M/42.6M [00:00<00:00, 142MB/s] 
100% 42.6M/42.6M [00:00<00:00, 150MB/s]


In [ ]:
! mkdir brazilian-ecommerce

In [ ]:
! unzip brazilian-ecommerce.zip -d brazilian-ecommerce

Archive:  brazilian-ecommerce.zip
  inflating: brazilian-ecommerce/olist_customers_dataset.csv  
  inflating: brazilian-ecommerce/olist_geolocation_dataset.csv  
  inflating: brazilian-ecommerce/olist_order_items_dataset.csv  
  inflating: brazilian-ecommerce/olist_order_payments_dataset.csv  
  inflating: brazilian-ecommerce/olist_order_reviews_dataset.csv  
  inflating: brazilian-ecommerce/olist_orders_dataset.csv  
  inflating: brazilian-ecommerce/olist_products_dataset.csv  
  inflating: brazilian-ecommerce/olist_sellers_dataset.csv  
  inflating: brazilian-ecommerce/product_category_name_translation.csv  


In [ ]:
#Importa as planilhas para um dataframe
pd = spark.read.csv('brazilian-ecommerce/olist_products_dataset.csv',inferSchema=True, header=True, sep=',') 
oid = spark.read.csv('brazilian-ecommerce/olist_order_items_dataset.csv',inferSchema=True, header=True, sep=',')
pyd = spark.read.csv('brazilian-ecommerce/olist_order_payments_dataset.csv',inferSchema=True, header=True, sep=',')
od = spark.read.csv('brazilian-ecommerce/olist_orders_dataset.csv',inferSchema=True, header=True, sep=',') 
sd = spark.read.csv('brazilian-ecommerce/olist_sellers_dataset.csv',inferSchema=True, header=True, sep=',')
cd = spark.read.csv('brazilian-ecommerce/olist_customers_dataset.csv',inferSchema=True, header=True, sep=',')
gd = spark.read.csv('brazilian-ecommerce/olist_geolocation_dataset.csv',inferSchema=True, header=True, sep=',')

In [ ]:
pd.createOrReplaceTempView("pd")
oid.createOrReplaceTempView("oid")
pyd.createOrReplaceTempView("pyd")
od.createOrReplaceTempView("od")
sd.createOrReplaceTempView("sd")
cd.createOrReplaceTempView("cd")
gd.createOrReplaceTempView("gd")

In [ ]:
dataset_list = [pd, oid, pyd, od, sd, cd, gd]

In [ ]:
def show_Dataset(datasetlist):
  for dataset in datasetlist:  
    print(datasetlist)
    dataset.show()

In [ ]:
show_Dataset(dataset_list)

[DataFrame[product_id: string, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int], DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double], DataFrame[order_id: string, payment_sequential: int, payment_type: string, payment_installments: int, payment_value: double], DataFrame[order_id: string, customer_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp], DataFrame[seller_id: string, seller_zip_code_prefix: int, seller_city: string, seller_state: string], DataFrame[customer_id: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: stri

In [ ]:
import numpy as np

In [ ]:
#usando sql para selecionar os dados 
queryregiao=spark.sql("select concat('Brasil - ', sd.seller_state) as Estado, count (DISTINCT od.customer_id) as Clientes, count (DISTINCT sd.seller_id) as Vendedores, round(sum(cast(pyd.payment_value as decimal)),2) as Faturamento from od join oid on oid.order_id=od.order_id join sd on oid.seller_id=sd.seller_id join pyd on od.order_id=pyd.order_id join pd on oid.product_id=pd.product_id group by sd.seller_state ")

In [ ]:
queryregiao.show(100,False)

+-----------+--------+----------+-----------+
|Estado     |Clientes|Vendedores|Faturamento|
+-----------+--------+----------+-----------+
|Brasil - SC|3667    |190       |886765     |
|Brasil - RO|14      |2         |5554       |
|Brasil - PI|12      |1         |3184       |
|Brasil - GO|463     |40        |112204     |
|Brasil - MT|137     |4         |23643      |
|Brasil - SP|70188   |1849      |13370891   |
|Brasil - PB|36      |6         |20609      |
|Brasil - ES|318     |23        |78828      |
|Brasil - RS|1989    |129       |560278     |
|Brasil - MS|49      |5         |10821      |
|Brasil - MG|7930    |244       |1564744    |
|Brasil - PA|8       |1         |1393       |
|Brasil - BA|569     |19        |367910     |
|Brasil - SE|9       |2         |2465       |
|Brasil - PE|406     |9         |124883     |
|Brasil - CE|91      |13        |25846      |
|Brasil - RN|51      |5         |13553      |
|Brasil - RJ|4353    |171       |1098261    |
|Brasil - MA|392     |1         |5

In [ ]:
import pyspark.pandas as ps
import pandas as pf
analise=queryregiao.to_pandas_on_spark()
analisecluster=queryregiao.to_pandas_on_spark()


spark-3.3.0-bin-hadoop2/python/pyspark/sql/dataframe.py:3315: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  FutureWarning,


In [ ]:
#normalizando os dados para os clusters
query_normalized_clientes=(analisecluster['Clientes'] - analisecluster['Clientes'].min()) / (analisecluster['Clientes'].max() - analisecluster['Clientes'].min())
query_normalized_vendedores=(analisecluster['Vendedores'] - analisecluster['Vendedores'].min()) / (analisecluster['Vendedores'].max() - analisecluster['Vendedores'].min())
query_normalized_faturamento=(analisecluster['Faturamento'] - analisecluster['Faturamento'].min()) / (analisecluster['Faturamento'].max() - analisecluster['Faturamento'].min())

In [ ]:
analisecluster['Clientes']=query_normalized_clientes
analisecluster['Vendedores']=query_normalized_vendedores
analisecluster['Faturamento']=query_normalized_faturamento

In [ ]:
analisecluster

,Estado,Clientes,Vendedores,Faturamento
0,Brasil - SC,0.052232,0.102273,0.066300
1,Brasil - RO,0.000185,0.000541,0.000393
2,Brasil - PI,0.000157,0.000000,0.000216
3,Brasil - GO,0.006582,0.021104,0.008369
4,Brasil - MT,0.001938,0.001623,0.001746
5,Brasil - SP,1.000000,1.000000,1.000000
6,Brasil - PB,0.000499,0.002706,0.001519
7,Brasil - ES,0.004517,0.011905,0.005873
8,Brasil - RS,0.028324,0.069264,0.041881
9,Brasil - MS,0.000684,0.002165,0.000787


In [ ]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=["Clientes","Vendedores","Faturamento"], outputCol="features")
new_df = vecAssembler.transform(analisecluster.to_spark())
new_df.show()

spark-3.3.0-bin-hadoop2/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+-----------+--------------------+--------------------+-----------+--------------------+
|     Estado|            Clientes|          Vendedores|Faturamento|            features|
+-----------+--------------------+--------------------+-----------+--------------------+
|Brasil - SC| 0.05223189479533247| 0.10227272727272728|0.066299612|[0.05223189479533...|
|Brasil - RO|1.852194850898314...|5.411255411255411E-4| 3.92952E-4|[1.85219485089831...|
|Brasil - PI|1.567241796913958...|                 0.0| 2.15697E-4|[1.56724179691395...|
|Brasil - GO|0.006582415547038626|0.021103896103896104|0.008369413|[0.00658241554703...|
|Brasil - MT|0.001937680767093...|0.001623376623376...|0.001745847|[0.00193768076709...|
|Brasil - SP|                 1.0|                 1.0|        1.0|       [1.0,1.0,1.0]|
|Brasil - PB|4.986678444726232E-4|0.002705627705627...|0.001518931|[4.98667844472623...|
|Brasil - ES|0.004516505905652...|0.011904761904761904|0.005873188|[0.00451650590565...|
|Brasil - RS|0.028324

In [ ]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=3, seed=1)  # 3 clusters here
model = kmeans.fit(new_df.select('features'))

In [ ]:
from pyspark.sql.functions import format_number 

query_kmeans = model.transform(new_df)
query_kmeans = query_kmeans.drop('features')
query_kmeans = query_kmeans.withColumnRenamed("prediction", "cluster")
query_kmeans = query_kmeans.withColumn("Clientes",format_number('Clientes',5))
query_kmeans = query_kmeans.withColumn("Vendedores",format_number('Vendedores',5))
query_kmeans = query_kmeans.withColumn("Faturamento",format_number('Faturamento',5))
query_kmeans.show() 

+-----------+--------+----------+-----------+-------+
|     Estado|Clientes|Vendedores|Faturamento|cluster|
+-----------+--------+----------+-----------+-------+
|Brasil - SC| 0.05223|   0.10227|    0.06630|      2|
|Brasil - RO| 0.00019|   0.00054|    0.00039|      0|
|Brasil - PI| 0.00016|   0.00000|    0.00022|      0|
|Brasil - GO| 0.00658|   0.02110|    0.00837|      0|
|Brasil - MT| 0.00194|   0.00162|    0.00175|      0|
|Brasil - SP| 1.00000|   1.00000|    1.00000|      1|
|Brasil - PB| 0.00050|   0.00271|    0.00152|      0|
|Brasil - ES| 0.00452|   0.01190|    0.00587|      0|
|Brasil - RS| 0.02832|   0.06926|    0.04188|      0|
|Brasil - MS| 0.00068|   0.00216|    0.00079|      0|
|Brasil - MG| 0.11297|   0.13149|    0.11701|      2|
|Brasil - PA| 0.00010|   0.00000|    0.00008|      0|
|Brasil - BA| 0.00809|   0.00974|    0.02749|      0|
|Brasil - SE| 0.00011|   0.00054|    0.00016|      0|
|Brasil - PE| 0.00577|   0.00433|    0.00932|      0|
|Brasil - CE| 0.00128|   0.0

In [ ]:
print(analise["Clientes"].sum()) 

99182


In [ ]:
print(analise["Vendedores"].sum()) 

3095


In [ ]:
print(analise["Faturamento"].sum())

20309206


In [ ]:
analise

,Estado,Clientes,Vendedores,Faturamento
0,Brasil - SC,3667,190,886765
1,Brasil - RO,14,2,5554
2,Brasil - PI,12,1,3184
3,Brasil - GO,463,40,112204
4,Brasil - MT,137,4,23643
5,Brasil - SP,70188,1849,13370891
6,Brasil - PB,36,6,20609
7,Brasil - ES,318,23,78828
8,Brasil - RS,1989,129,560278
9,Brasil - MS,49,5,10821


In [ ]:
analise.describe()

,Clientes,Vendedores,Faturamento
count,23.000000,23.000000,2.300000e+01
mean,4312.260870,134.565217,8.830090e+05
std,14551.512515,385.498240,2.773278e+06
min,1.000000,1.000000,3.000000e+02
25%,14.000000,2.000000,5.554000e+03
50%,318.000000,9.000000,5.199600e+04
75%,1989.000000,129.000000,5.602780e+05
max,70188.000000,1849.000000,1.337089e+07


In [ ]:
def analisar_coluna(coluna_nome): #criamos uma classefuncao para analisar cada coluna e reduzir o codigo
        coluna = coluna_nome
        media = analise[coluna_nome].mean()
        mediana = analise[coluna_nome].median()
        primeiro_quartil = analise[coluna_nome].quantile(.25)
        segundo_quartil = analise[coluna_nome].quantile(.50)
        terceiro_quartil = analise[coluna_nome].quantile(.75)
        quarto_quartil = analise[coluna_nome].quantile(1.0)
        desvio_padrao = analise[coluna_nome].std()
        maximo = analise[coluna_nome].max()
        minimo = analise[coluna_nome].min()
        Analise = (coluna, media, mediana, primeiro_quartil, segundo_quartil, terceiro_quartil, quarto_quartil, desvio_padrao, maximo, minimo)
        return Analise

In [ ]:
analise_Clientes = analisar_coluna("Clientes")

In [ ]:
analise_Vendedores = analisar_coluna("Vendedores")

In [ ]:
analise_Faturamento = analisar_coluna("Faturamento")

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType

In [ ]:
data = [(analise_Clientes), (analise_Vendedores), (analise_Faturamento)]

columns = StructType([ \
StructField("coluna",StringType(),True), \
StructField("media",StringType(),True), \
StructField("mediana",StringType(),True), \
StructField("primeiro_quartil",StringType(),True), \
StructField("segundo_quartil", StringType(), True), \
StructField("terceiro_quartil", StringType(), True), \
StructField("quarto_quartil", StringType(), True), \
StructField("desvio_padrao", StringType(), True), \
StructField("maximo", StringType(), True), \
StructField("minimo", StringType(), True) \
  ])

In [ ]:
spark2 = SparkSession.builder.master("local[*]").appName("AnalisExploratória").getOrCreate()
sdf = spark2.createDataFrame(data=data,schema = columns)

In [ ]:
sdf.createOrReplaceTempView("sdf")

In [ ]:
queryanalise=spark.sql("select coluna as coluna, round(media,2) as media,mediana,primeiro_quartil,segundo_quartil,terceiro_quartil,round(cast(quarto_quartil as decimal),2) as quarto_quartil,round(desvio_padrao,2) as desvio_padrao,maximo,minimo from sdf")

In [ ]:
export_csv = queryanalise
export_csv.show(100,False)

+-----------+---------+-------+----------------+---------------+----------------+--------------+-------------+--------+------+
|coluna     |media    |mediana|primeiro_quartil|segundo_quartil|terceiro_quartil|quarto_quartil|desvio_padrao|maximo  |minimo|
+-----------+---------+-------+----------------+---------------+----------------+--------------+-------------+--------+------+
|Clientes   |4312.26  |318.0  |14.0            |318.0          |1989.0          |70188         |14551.51     |70188   |1     |
|Vendedores |134.57   |9.0    |2.0             |9.0            |129.0           |1849          |385.5        |1849    |1     |
|Faturamento|883008.96|51996.0|5554.0          |51996.0        |560278.0        |13370891      |2773278.07   |13370891|300   |
+-----------+---------+-------+----------------+---------------+----------------+--------------+-------------+--------+------+



In [ ]:
export_csv.toPandas().to_csv("OlistAnaliseExploratoria.csv", index=False, sep=':')

In [ ]:
from google.colab import files
files.download("OlistAnaliseExploratoria.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
export_csv = queryregiao
export_csv.show(100,False)

+-----------+--------+----------+-----------+
|Estado     |Clientes|Vendedores|Faturamento|
+-----------+--------+----------+-----------+
|Brasil - SC|3667    |190       |886765     |
|Brasil - RO|14      |2         |5554       |
|Brasil - PI|12      |1         |3184       |
|Brasil - GO|463     |40        |112204     |
|Brasil - MT|137     |4         |23643      |
|Brasil - SP|70188   |1849      |13370891   |
|Brasil - PB|36      |6         |20609      |
|Brasil - ES|318     |23        |78828      |
|Brasil - RS|1989    |129       |560278     |
|Brasil - MS|49      |5         |10821      |
|Brasil - MG|7930    |244       |1564744    |
|Brasil - PA|8       |1         |1393       |
|Brasil - BA|569     |19        |367910     |
|Brasil - SE|9       |2         |2465       |
|Brasil - PE|406     |9         |124883     |
|Brasil - CE|91      |13        |25846      |
|Brasil - RN|51      |5         |13553      |
|Brasil - RJ|4353    |171       |1098261    |
|Brasil - MA|392     |1         |5

In [ ]:
export_csv.toPandas().to_csv("OlistAnaliseregiao.csv", index=False, sep=':')

In [ ]:
files.download("OlistAnaliseregiao.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
export_csv = query_kmeans
export_csv.show(100,False)

+-----------+--------+----------+-----------+-------+
|Estado     |Clientes|Vendedores|Faturamento|cluster|
+-----------+--------+----------+-----------+-------+
|Brasil - SC|0.05223 |0.10227   |0.06630    |2      |
|Brasil - RO|0.00019 |0.00054   |0.00039    |0      |
|Brasil - PI|0.00016 |0.00000   |0.00022    |0      |
|Brasil - GO|0.00658 |0.02110   |0.00837    |0      |
|Brasil - MT|0.00194 |0.00162   |0.00175    |0      |
|Brasil - SP|1.00000 |1.00000   |1.00000    |1      |
|Brasil - PB|0.00050 |0.00271   |0.00152    |0      |
|Brasil - ES|0.00452 |0.01190   |0.00587    |0      |
|Brasil - RS|0.02832 |0.06926   |0.04188    |0      |
|Brasil - MS|0.00068 |0.00216   |0.00079    |0      |
|Brasil - MG|0.11297 |0.13149   |0.11701    |2      |
|Brasil - PA|0.00010 |0.00000   |0.00008    |0      |
|Brasil - BA|0.00809 |0.00974   |0.02749    |0      |
|Brasil - SE|0.00011 |0.00054   |0.00016    |0      |
|Brasil - PE|0.00577 |0.00433   |0.00932    |0      |
|Brasil - CE|0.00128 |0.0064

In [ ]:
export_csv.toPandas().to_csv("OlistClusters.csv", index=False , sep=':')

In [ ]:
files.download("OlistClusters.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>